In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import sys
import glob

sys.path.append("../lib/")
import eyegaze as eg

In [ ]:
# ========== 設定 ==========
group_letter = "B"
participant_id = "P001"
phase = "pre"

# パス設定
base_dir = f"../../data/input/{group_letter}/{participant_id}/{phase}"
working_dir = f"../../data/working/{group_letter}/{participant_id}/{phase}"
output_dir = f"../../data/output/{group_letter}/{participant_id}/{phase}"

# eye_trackingディレクトリ（背景画像用）
eye_tracking_base = os.path.join(base_dir, "eye_tracking")
timestamp_dirs = sorted([d for d in os.listdir(eye_tracking_base) 
                         if os.path.isdir(os.path.join(eye_tracking_base, d))])
eye_tracking_dir = os.path.join(eye_tracking_base, timestamp_dirs[-1])

print(f"Group: {group_letter}, Participant: {participant_id}, Phase: {phase}")
print(f"Eye tracking dir: {eye_tracking_dir}")

In [ ]:
# 座標ファイルとfixationファイルの対応を作成
coordinates_dir = os.path.join(base_dir, "coordinates")
fixation_dir = os.path.join(working_dir, "fixation")

# 座標ファイル一覧
coord_files = sorted(glob.glob(os.path.join(coordinates_dir, "question_*.json")))
print(f"Found {len(coord_files)} coordinate files")

# passage_id と image_number の対応
# pre_01 -> 003, pre_02 -> 004, ...
segments_info = []
for coord_file in coord_files:
    coords = eg.loadCoordinates(coord_file)
    passage_id = coords['coordinates']['passage_id']
    img_num = eg.passageIdToImageNumber(passage_id)
    
    fixation_file = os.path.join(fixation_dir, f"{img_num}.csv")
    image_file = os.path.join(eye_tracking_dir, f"{img_num}_back.png")
    
    segments_info.append({
        'passage_id': passage_id,
        'image_number': img_num,
        'coord_file': coord_file,
        'fixation_file': fixation_file,
        'image_file': image_file,
        'fixation_exists': os.path.exists(fixation_file)
    })

for seg in segments_info:
    status = "OK" if seg['fixation_exists'] else "MISSING"
    print(f"  {seg['passage_id']} ({seg['image_number']}): {status}")

In [ ]:
# サンプル: 1つのセグメントでAOI分析
sample_seg = segments_info[0]
print(f"Analyzing: {sample_seg['passage_id']}")

# 座標データ読み込み
coords = eg.loadCoordinates(sample_seg['coord_file'])

# AOI抽出（全レベル）
aois = eg.extractAOIs(coords, levels=["paragraph", "sentence", "word", "choice", "question"])
print(f"Extracted {len(aois)} AOIs")

# レベル別集計
for level in ["paragraph", "sentence", "word", "choice", "question"]:
    count = len([a for a in aois if a['level'] == level])
    print(f"  {level}: {count}")

In [ ]:
# fixationデータ読み込み
fixations = pd.read_csv(sample_seg['fixation_file'])
print(f"Loaded {len(fixations)} fixations")
print(fixations.head())

In [ ]:
# AOIマッチング
matched = eg.matchFixationsToAOIs(fixations, aois)

# DataFrameに変換して確認
matched_df = pd.DataFrame(matched)
print(f"Matched {len(matched_df)} fixations")
print("\nSample (first 10 rows):")
print(matched_df[['timestamp', 'x', 'y', 'duration', 'sentence_id', 'word_text']].head(10))

In [ ]:
# 文レベルの統計
sent_stats = eg.computeAOIStatistics(matched, level="sentence")
print("=== Sentence-level Statistics ===")
print(sent_stats[['aoi_id', 'text', 'total_duration', 'fixation_count', 'revisits']])

In [ ]:
# 選択肢レベルの統計
choice_stats = eg.computeAOIStatistics(matched, level="choice")
print("=== Choice-level Statistics ===")
print(choice_stats[['aoi_id', 'text', 'total_duration', 'fixation_count', 'revisits']])

In [ ]:
# 可視化: 文レベル
eg.plotAOIWithGaze(
    sample_seg['image_file'],
    aois,
    fixations,
    level="sentence"
)

In [ ]:
# 可視化: 選択肢レベル
eg.plotAOIWithGaze(
    sample_seg['image_file'],
    aois,
    fixations,
    level="choice"
)

In [ ]:
# 全セグメントの一括処理
all_matched = []
all_stats = []

for seg in segments_info:
    if not seg['fixation_exists']:
        print(f"Skipping {seg['passage_id']}: fixation file not found")
        continue
    
    print(f"Processing {seg['passage_id']}...")
    
    # 座標とfixation読み込み
    coords = eg.loadCoordinates(seg['coord_file'])
    aois = eg.extractAOIs(coords)
    fixations = pd.read_csv(seg['fixation_file'])
    
    # マッチング
    matched = eg.matchFixationsToAOIs(fixations, aois)
    matched_df = pd.DataFrame(matched)
    matched_df['passage_id'] = seg['passage_id']
    all_matched.append(matched_df)
    
    # 統計（文レベル）
    sent_stats = eg.computeAOIStatistics(matched, level="sentence")
    sent_stats['passage_id'] = seg['passage_id']
    all_stats.append(sent_stats)
    
    # 可視化保存
    viz_dir = os.path.join(output_dir, "aoi_visualization")
    os.makedirs(viz_dir, exist_ok=True)
    eg.plotAOIWithGaze(
        seg['image_file'],
        aois,
        fixations,
        level="sentence",
        save_path=os.path.join(viz_dir, f"{seg['image_number']}_sentence.png")
    )

print("\nDone!")

In [ ]:
# 結果をCSVに保存
if all_matched:
    # 時系列データ
    all_matched_df = pd.concat(all_matched, ignore_index=True)
    aoi_fixations_dir = os.path.join(output_dir, "aoi_fixations")
    os.makedirs(aoi_fixations_dir, exist_ok=True)
    all_matched_df.to_csv(os.path.join(aoi_fixations_dir, "all_fixations.csv"), index=False)
    print(f"Saved {len(all_matched_df)} fixation records")
    
    # 統計サマリー
    all_stats_df = pd.concat(all_stats, ignore_index=True)
    all_stats_df.to_csv(os.path.join(output_dir, "aoi_statistics.csv"), index=False)
    print(f"Saved statistics for {len(all_stats_df)} AOIs")

In [ ]:
# 統計サマリー表示
if all_stats:
    all_stats_df = pd.concat(all_stats, ignore_index=True)
    
    print("=== Overall Statistics ===")
    print(f"Total sentences analyzed: {len(all_stats_df)}")
    print(f"Total fixation time: {all_stats_df['total_duration'].sum():.2f}s")
    print(f"Mean fixation time per sentence: {all_stats_df['total_duration'].mean():.3f}s")
    print("\nTop 10 most viewed sentences:")
    print(all_stats_df.nlargest(10, 'total_duration')[['passage_id', 'aoi_id', 'text', 'total_duration', 'fixation_count']])